<a href="https://colab.research.google.com/github/BatXprO/batxpro.githib.io/blob/main/pdf_extration_and_ocr_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!apt-get install -y poppler-utils
!pip install pdf2image pillow pytesseract pandas openpyxl
!sudo apt install tesseract-ocr # Install Tesseract OCR engine
!sudo apt install libtesseract-dev # Install Tesseract development files


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.

In [ ]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import pandas as pd
import re
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Define OCR-based extraction function
def extract_contacts_from_pdf(pdf_path):
    # Lists to store results
    emails = []
    phones = []
    names = []

    # Convert PDF pages to images
    pages = convert_from_path(pdf_path, dpi=300)  # Higher dpi for better OCR accuracy

    for page_num, page in enumerate(pages):
        # Perform OCR on each page
        text = pytesseract.image_to_string(page)

        # Define regex patterns for email, phone, and name
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        phone_pattern = r'(\b\d{3}[-.\s]??\d{3}[-.\s]??\d{4}\b|\b\d{10}\b)'

        # Extract emails
        emails_found = re.findall(email_pattern, text)
        emails.extend(emails_found)

        # Extract phone numbers
        phones_found = re.findall(phone_pattern, text)
        phones.extend(phones_found)

        # Optional: Extract names (based on keywords like "Name" or "Owner" in the text)
        name_pattern = r'(Name[:\s]*[A-Za-z\s]+|Owner[:\s]*[A-Za-z\s]+)'
        names_found = re.findall(name_pattern, text)
        names.extend([name.split(':')[1].strip() for name in names_found if ':' in name])

    # Create a DataFrame to organize the extracted information
    # Find the maximum length among the lists
    max_len = max(len(emails), len(phones), len(names))

    # Pad the shorter lists with None to make them equal in length
    emails.extend([None] * (max_len - len(emails)))
    phones.extend([None] * (max_len - len(phones)))
    names.extend([None] * (max_len - len(names)))

    df = pd.DataFrame({
        'Email': emails,
        'Phone': phones,
        'Name': names  # Now all lists have the same length
    })

    # Drop duplicate entries, if any
    df = df.drop_duplicates().reset_index(drop=True)

    return df

def save_to_excel(df, output_path):
    # Create a new Excel workbook and worksheet
    wb = Workbook()
    ws = wb.active
    ws.title = "Contacts"

    # Write the DataFrame to the worksheet
    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)

    # Adjust column widths based on the max length in each column
    for col in ws.columns:
        max_length = 0
        col_letter = col[0].column_letter  # Get column name
        for cell in col:
            try:
                max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = (max_length + 2)  # Add some padding
        ws.column_dimensions[col_letter].width = adjusted_width

    # Save the Excel file
    wb.save(output_path)

# Specify the path to your PDF file
pdf_path = "/content/Exhibition-Catalogue-2023.pdf"
output_path = "/content/extracted_contacts.xlsx"

# Run the extraction and save to Excel
contacts_df = extract_contacts_from_pdf(pdf_path)
save_to_excel(contacts_df, output_path)

# Display a success message with the file path
print(f"Data successfully saved to {output_path}")


Data successfully saved to /content/extracted_contacts.xlsx


In [ ]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>